In [2]:
# first, test generative model on one dimensional graphs
import sys
sys.path.append('./HCM') # outdated due to abstraction learning implementation
import pandas as pd
import seaborn as sns
import numpy as np
from Learning import *
from CG1 import *

In [4]:
import pandas as pd
df = pd.read_csv('/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/data.csv')

dfm = {}  # model dataframe
dfm['blockcollect'] = []
dfm['ID'] = []
dfm['condition'] = []
dfm['correctcollect'] = []
dfm['p'] = []
dfm['trialcollect'] = []
dfm['recall_likelihood'] = [] # the likelihood of subjects recalling the exact piece of sequence, given the representation learned so far


column_names = ['ID', 'condition', 'recallcollect', 'instructioncollect','trialcollect', 'correctcollect']
dfs = pd.DataFrame(columns=column_names)  # Note that there are now row data inserted.



seql = 12
len_train = 40
len_test = 8

def convert_sequence(seq,keyassignment):
    keyassignment = list(keyassignment)
    ka = [keyassignment[2],keyassignment[7],keyassignment[12],keyassignment[17],keyassignment[22],keyassignment[27]]
    proj_seq = [ka.index(item)+ 1 for item in seq]
    return proj_seq

def convert_sequence_backward_to_key(seq, keyassignment):
    keyassignment = list(keyassignment)
    inst_seq = []
    ka = [keyassignment[2], keyassignment[7], keyassignment[12], keyassignment[17], keyassignment[22],
          keyassignment[27]]
    for i in range(0, 12):
        inst_seq.append(ka[int(seq[i,0,0])-1])
    return inst_seq

def calculate_prob(chunk_record, cg):
    p = 1
    for key in list(chunk_record.keys()):# key is the encoding time
        p = p*cg.chunks[chunk_record[key][0][0]].count/np.sum([item.count for item in cg.chunks])
    return p
    

for sub in np.unique(list(df['ID'])):
    # initialize chunking part with specified parameters
    cg = CG1(DT=0.1, theta=0.996)
    for trial in range(1, len_train + 3*len_test+ 1):
        ins_seq = df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'instructioncollect']
        subj_recall =  df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'recallcollect']
        keyassignment = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'keyassignment'])[0]
        condition = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'condition'])[0]
        block = list(df[(df['ID'] == sub)].iloc[(trial-1)*seql:trial*seql, :][
            'blockcollect'])[0]
        proj_seq = convert_sequence(list(ins_seq),keyassignment)
        #print(proj_seq)
        proj_seq = np.array(proj_seq).reshape([-1, 1, 1])
        
    
        cg, chunkrecord = hcm_learning(proj_seq, cg)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recalled_seq,ps = recall(cg, firstitem=proj_seq[0,0,0])   
        keypress_recalled_seq = convert_sequence_backward_to_key(recalled_seq,keyassignment)    
        
        p_seq = np.prod(ps)# evaluate the probability of a sequence
        
        
        proj_recalled_seq = convert_sequence(list(subj_recall),keyassignment)
        proj_recalled_seq = np.array(proj_recalled_seq).reshape([-1, 1, 1])
        cg, chunkrecord = hcm_learning(proj_recalled_seq, cg, learn = False, chunk = False)  # with the rational chunk models, rational_chunk_all_info(seq, cg)
        recall_likelihood = calculate_prob(chunkrecord, cg)# evaluate the probability of a sequence
        print(recall_likelihood)
        
        dfm['blockcollect'].append(block)
        dfm['ID'].append(sub)
        dfm['condition'].append(condition)
        dfm['correctcollect'].append(acc_eval1d(recalled_seq, proj_seq))
        dfm['p'].append(p_seq)
        dfm['trialcollect'].append(trial)
        dfm['recall_likelihood'].append(recall_likelihood)
        
        ins_list = ins_seq.tolist()
        print(keypress_recalled_seq, ins_list)
        for i in range(0, 12):
            if keypress_recalled_seq[i] == ins_list[i]:
                correctcollect = 1
            else: correctcollect = 0
            dfs = dfs.append({'ID': sub,
                            'condition': condition,
                            'keyassignment': keyassignment,
                            'recallcollect': keypress_recalled_seq[i],
                            'trialcollect': trial,
                            'instructioncollect': ins_list[i],
                            'correctcollect': correctcollect}, ignore_index=True)

dfm = pd.DataFrame.from_dict(dfm)
csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/simulation_data_model_transition_recall_leison_variable_motif.csv'
dfm.to_csv(csv_save_directory, index=False, header=True)


dfs_csv_save_directory = '/Users/swu/Desktop/research/motif_learning/data/simonsays_ex2/simulation_data_model_transition_recall_leison_variable_motif_individualkey.csv'
dfs.to_csv(dfs_csv_save_directory, index=False, header=True)

4.375375182304919e-10
accuracy is:  0.3333333333333333
['J', 'D', 'S', 'S', 'L', 'L', 'L', 'L', 'L', 'L', 'J', 'D'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
6.078768801064287e-10
accuracy is:  0.6666666666666666
['J', 'D', 'S', 'L', 'L', 'L', 'L', 'J', 'S', 'L', 'L', 'J'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
4.5863180177297153e-10
accuracy is:  0.5
['J', 'D', 'S', 'L', 'L', 'L', 'K', 'J', 'D', 'S', 'L', 'L'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
7.967203925213008e-11
accuracy is:  0.4166666666666667
['J', 'D', 'S', 'L', 'L', 'L', 'S', 'J', 'J', 'D', 'S', 'S'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
1.2010287148567842e-08
accuracy is:  0.5833333333333334
['J', 'D', 'S', 'L', 'L', 'J', 'D', 'S', 'L', 'L', 'L', 'K'] ['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'F', 'L', 'J']
1.1039139644845833e-07
accuracy is:  0.75
['J', 'D', 'S', 'L', 'L', 'K', 'J', 'S', 'S', 'S', 'S', 'L'] ['J', 'D', 'S', 'L', 'L',

3.7285019015520915e-08
accuracy is:  0.3333333333333333
['S', 'D', 'L', 'J', 'S', 'L', 'L', 'K', 'S', 'L', 'L', 'D'] ['S', 'D', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'K', 'S', 'J']
2.805516484810974e-13
accuracy is:  0.25
['S', 'S', 'F', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'S'] ['S', 'K', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'F', 'S', 'J']
9.973142801280234e-12
accuracy is:  0.75
['S', 'D', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'J', 'S'] ['S', 'D', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'K', 'S', 'J']
4.590124166958294e-10
accuracy is:  0.8333333333333334
['S', 'F', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'J', 'S'] ['S', 'F', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'S', 'J']
2.4433812911193005e-11
accuracy is:  0.5833333333333334
['S', 'D', 'L', 'J', 'J', 'D', 'S', 'L', 'L', 'K', 'J', 'S'] ['S', 'K', 'L', 'J', 'J', 'F', 'S', 'L', 'L', 'D', 'S', 'J']
2.176314297047722e-11
accuracy is:  0.25
['S', 'L', 'L', 'D', 'S', 'F', 'S', 'J', 'D', 'S', 'L', 'L'] ['S', 'F', 'L', 'J', 'J', 'K', 'S', 'L'

3.0112634573027075e-07
accuracy is:  0.75
['K', 'D', 'S', 'L', 'S', 'F', 'K', 'L', 'L', 'J', 'S', 'K'] ['K', 'F', 'S', 'L', 'S', 'J', 'K', 'L', 'L', 'D', 'S', 'K']
1.8539426194272044e-08
accuracy is:  0.3333333333333333
['K', 'L', 'L', 'J', 'S', 'K', 'F', 'K', 'L', 'L', 'J', 'K'] ['K', 'J', 'S', 'L', 'S', 'F', 'K', 'L', 'L', 'D', 'S', 'K']
6.490855329732618e-07
accuracy is:  0.8333333333333334
['K', 'D', 'S', 'L', 'S', 'F', 'K', 'L', 'L', 'J', 'S', 'K'] ['K', 'F', 'S', 'L', 'S', 'D', 'K', 'L', 'L', 'J', 'S', 'K']
7.556817855292899e-05
accuracy is:  0.3333333333333333
['K', 'L', 'L', 'D', 'S', 'K', 'F', 'K', 'L', 'L', 'J', 'K'] ['K', 'F', 'S', 'L', 'S', 'J', 'K', 'L', 'L', 'D', 'S', 'K']
1.0099909297049362e-12
accuracy is:  0.16666666666666666
['S', 'D', 'K', 'L', 'L', 'K', 'D', 'K', 'L', 'L', 'K', 'J'] ['S', 'J', 'L', 'K', 'L', 'F', 'K', 'S', 'K', 'D', 'L', 'S']
3.906576105026377e-13
accuracy is:  0.16666666666666666
['S', 'L', 'S', 'D', 'K', 'L', 'L', 'J', 'S', 'K', 'F', 'S'] ['S', 'F

4.016137917648799e-09
accuracy is:  0.16666666666666666
['K', 'S', 'F', 'K', 'F', 'F', 'K', 'F', 'S', 'F', 'K', 'F'] ['K', 'L', 'J', 'F', 'J', 'S', 'F', 'K', 'F', 'D', 'K', 'J']
7.3655462121177755e-09
accuracy is:  0.5
['K', 'S', 'F', 'K', 'F', 'L', 'F', 'K', 'F', 'K', 'L', 'F'] ['K', 'S', 'J', 'F', 'J', 'L', 'F', 'K', 'F', 'D', 'K', 'J']
1.851105008637584e-08
accuracy is:  0.3333333333333333
['K', 'J', 'D', 'F', 'K', 'F', 'L', 'J', 'F', 'J', 'K', 'S'] ['K', 'D', 'J', 'F', 'J', 'S', 'F', 'K', 'F', 'L', 'K', 'J']
3.1430028634295507e-09
accuracy is:  1.0
['K', 'S', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'L', 'K', 'J'] ['K', 'S', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'L', 'K', 'J']
4.468393453541613e-10
accuracy is:  0.16666666666666666
['K', 'S', 'K', 'J', 'D', 'F', 'K', 'F', 'L', 'F', 'K', 'F'] ['K', 'L', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'S', 'K', 'J']
5.144091602865403e-12
accuracy is:  0.8333333333333334
['K', 'S', 'J', 'F', 'J', 'D', 'F', 'K', 'F', 'L', 'K', 'J'] ['K', 'D', 'J', 'F', 'J',

3.736050510209506e-09
accuracy is:  0.08333333333333333
['K', 'S', 'J', 'S', 'F', 'S', 'D', 'D', 'L', 'K', 'S', 'S'] ['K', 'J', 'S', 'D', 'D', 'F', 'K', 'S', 'S', 'L', 'K', 'D']
7.285648957217424e-10
accuracy is:  0.25
['K', 'S', 'J', 'K', 'D', 'F', 'S', 'D', 'D', 'J', 'S', 'D'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
2.380517054968328e-09
accuracy is:  0.5
['K', 'J', 'S', 'D', 'D', 'J', 'S', 'D', 'D', 'F', 'S', 'D'] ['K', 'J', 'S', 'D', 'D', 'F', 'K', 'S', 'S', 'L', 'K', 'D']
5.99419880864174e-09
accuracy is:  0.3333333333333333
['K', 'S', 'S', 'J', 'S', 'J', 'S', 'D', 'D', 'F', 'K', 'J'] ['K', 'J', 'S', 'D', 'D', 'L', 'K', 'S', 'S', 'F', 'K', 'D']
4.842675509447872e-09
accuracy is:  0.5833333333333334
['K', 'F', 'S', 'D', 'D', 'J', 'S', 'D', 'D', 'F', 'S', 'D'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
3.7632178917208124e-09
accuracy is:  0.25
['K', 'S', 'J', 'S', 'D', 'D', 'F', 'S', 'D', 'D', 'J', 'S'] ['K', 'F', 'S', 'D', 'D', 'L', 'K', 'S', 

...

1.1386547232204859e-05
accuracy is:  0.3333333333333333
['S', 'D', 'K', 'J', 'F', 'S', 'F', 'L', 'S', 'K', 'L', 'K'] ['S', 'L', 'K', 'F', 'K', 'D', 'F', 'S', 'F', 'J', 'S', 'K']
6.376323029895892e-05
accuracy is:  0.3333333333333333
['S', 'K', 'L', 'F', 'S', 'F', 'J', 'K', 'F', 'K', 'D', 'K'] ['S', 'D', 'K', 'F', 'K', 'L', 'F', 'S', 'F', 'J', 'S', 'K']
5.162217342551335e-10
accuracy is:  0.16666666666666666
['S', 'K', 'L', 'F', 'S', 'F', 'J', 'F', 'S', 'F', 'F', 'S'] ['S', 'J', 'K', 'F', 'K', 'L', 'F', 'S', 'F', 'D', 'S', 'K']
1.3070223798643658e-05
accuracy is:  0.16666666666666666
['S', 'K', 'L', 'S', 'D', 'K', 'F', 'K', 'L', 'S', 'K', 'L'] ['S', 'L', 'K', 'F', 'K', 'D', 'F', 'S', 'F', 'J', 'S', 'K']
1.0401432665420736e-05
accuracy is:  0.5833333333333334
['S', 'L', 'K', 'F', 'K', 'K', 'F', 'S', 'D', 'K', 'F', 'K'] ['S', 'D', 'K', 'F', 'K', 'L', 'F', 'S', 'F', 'J', 'S', 'K']
2.4064303864100556e-08
accuracy is:  0.16666666666666666
['S', 'K', 'L', 'K', 'F', 'K', 'J', 'K', 'L', 'F', 'S

1.927557138344228e-08
accuracy is:  0.3333333333333333
['L', 'K', 'K', 'J', 'L', 'D', 'S', 'K', 'S', 'F', 'S', 'J'] ['L', 'F', 'S', 'K', 'S', 'D', 'L', 'K', 'K', 'J', 'S', 'L']
5.749703342773625e-11
accuracy is:  0.5
['L', 'J', 'S', 'K', 'S', 'J', 'S', 'D', 'S', 'K', 'S', 'J'] ['L', 'D', 'S', 'K', 'S', 'J', 'L', 'K', 'K', 'F', 'S', 'L']
4.2428475650203017e-10
accuracy is:  0.5833333333333334
['L', 'J', 'S', 'K', 'S', 'D', 'S', 'K', 'S', 'D', 'S', 'K'] ['L', 'F', 'S', 'K', 'S', 'J', 'L', 'K', 'K', 'D', 'S', 'L']
2.3442903757415183e-11
accuracy is:  0.08333333333333333
['L', 'S', 'L', 'F', 'L', 'K', 'K', 'F', 'L', 'K', 'K', 'F'] ['L', 'J', 'S', 'K', 'S', 'F', 'L', 'K', 'K', 'D', 'S', 'L']
3.945658444504407e-07
accuracy is:  0.25
['L', 'K', 'K', 'J', 'S', 'K', 'S', 'K', 'S', 'J', 'L', 'K'] ['L', 'J', 'S', 'K', 'S', 'D', 'L', 'K', 'K', 'F', 'S', 'L']
2.0110869731880284e-11
accuracy is:  0.16666666666666666
['L', 'F', 'L', 'K', 'K', 'D', 'S', 'L', 'D', 'S', 'L', 'D'] ['L', 'D', 'S', 'K', 'S

7.493660056642841e-12
accuracy is:  0.25
['S', 'L', 'D', 'K', 'L', 'K', 'F', 'K', 'S', 'J', 'L', 'S'] ['S', 'F', 'K', 'L', 'K', 'D', 'L', 'S', 'L', 'J', 'K', 'S']
1.2564674632677307e-11
accuracy is:  0.3333333333333333
['S', 'D', 'K', 'L', 'K', 'J', 'S', 'K', 'K', 'F', 'L', 'K'] ['S', 'J', 'K', 'L', 'K', 'F', 'L', 'S', 'L', 'D', 'K', 'S']
4.277491387323073e-12
accuracy is:  0.25
['S', 'F', 'L', 'K', 'K', 'D', 'S', 'L', 'D', 'K', 'S', 'J'] ['S', 'F', 'K', 'L', 'K', 'J', 'L', 'S', 'L', 'D', 'K', 'S']
1.931139954514171e-11
accuracy is:  0.16666666666666666
['S', 'F', 'L', 'K', 'K', 'D', 'S', 'L', 'D', 'K', 'S', 'J'] ['S', 'D', 'K', 'L', 'K', 'F', 'L', 'S', 'L', 'J', 'K', 'S']
4.158783510876831e-12
accuracy is:  0.16666666666666666
['S', 'J', 'L', 'K', 'K', 'K', 'S', 'J', 'K', 'F', 'L', 'K'] ['S', 'F', 'K', 'L', 'K', 'D', 'L', 'S', 'L', 'J', 'K', 'S']
2.3963795213030512e-12
accuracy is:  0.25
['S', 'K', 'K', 'D', 'L', 'S', 'L', 'J', 'K', 'S', 'J', 'K'] ['S', 'J', 'K', 'L', 'K', 'D', 'L', '

6.684613644728847e-11
accuracy is:  0.16666666666666666
['K', 'S', 'D', 'L', 'S', 'K', 'S', 'F', 'K', 'D', 'K', 'J'] ['K', 'J', 'S', 'D', 'D', 'L', 'K', 'S', 'S', 'F', 'K', 'D']
3.744656352938887e-11
accuracy is:  0.16666666666666666
['K', 'S', 'D', 'L', 'S', 'K', 'S', 'F', 'S', 'D', 'D', 'J'] ['K', 'F', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'L', 'K', 'D']
1.848839255165544e-11
accuracy is:  0.3333333333333333
['K', 'J', 'S', 'D', 'L', 'K', 'S', 'D', 'L', 'S', 'K', 'S'] ['K', 'F', 'S', 'D', 'D', 'L', 'K', 'S', 'S', 'J', 'K', 'D']
1.2712266569594389e-11
accuracy is:  0.3333333333333333
['K', 'J', 'S', 'D', 'L', 'K', 'S', 'D', 'L', 'S', 'K', 'S'] ['K', 'L', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'F', 'K', 'D']
2.2050496722459038e-12
accuracy is:  0.08333333333333333
['K', 'D', 'K', 'J', 'S', 'D', 'F', 'K', 'D', 'K', 'J', 'K'] ['K', 'F', 'S', 'D', 'D', 'J', 'K', 'S', 'S', 'L', 'K', 'D']
2.135724541734705e-11
accuracy is:  0.3333333333333333
['K', 'D', 'F', 'K', 'D', 'K', 'J', 'S', 'D', 'L', 'K',

0.00011214360272108773
accuracy is:  0.9166666666666666
['K', 'F', 'S', 'J', 'S', 'L', 'J', 'K', 'J', 'D', 'S', 'J'] ['K', 'F', 'S', 'J', 'S', 'L', 'J', 'K', 'J', 'D', 'S', 'K']
3.277579507809903e-06
accuracy is:  0.8333333333333334
['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'L', 'S', 'K'] ['K', 'L', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'F', 'S', 'K']
0.00013790848861889733
accuracy is:  1.0
['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'L', 'S', 'K'] ['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'L', 'S', 'K']
9.947317121454106e-06
accuracy is:  0.5833333333333334
['K', 'F', 'S', 'J', 'S', 'D', 'J', 'D', 'S', 'J', 'S', 'L'] ['K', 'F', 'S', 'J', 'S', 'L', 'J', 'K', 'J', 'D', 'S', 'K']
9.396868901133794e-06
accuracy is:  0.75
['K', 'F', 'S', 'J', 'S', 'D', 'J', 'K', 'J', 'D', 'S', 'K'] ['K', 'D', 'S', 'J', 'S', 'F', 'J', 'K', 'J', 'L', 'S', 'K']
1.5924835686065247e-08
accuracy is:  0.3333333333333333
['K', 'D', 'S', 'D', 'S', 'J', 'S', 'D', 'S', 'J', 'S', 'L'] ['K', 'F', 'S', 'J', 'S',